# <center>Analyze and Visualize Data with Intake-ESM, XArray and hvPlot</center>
## <center>A Climate Data Use Case</center>

We will show here how to count the annual summer days for a particular geolocation of your choice using the results of a climate model, in particular, we can chose one of the historical or one of the shared socioeconomic pathway (ssp) experiments of the Coupled Model Intercomparison Project [CMIP6](https://pcmdi.llnl.gov/CMIP6/).

This Jupyter notebook is meant to run in the Jupyterhub server of the German Climate Computing Center [DKRZ](https://www.dkrz.de/) which is an [ESGF](https://esgf.llnl.gov/) repository that hosts 4 petabytes of CMIP6 data. Please, choose the Python 3 unstable kernel on the Kernel tab above, it contains all the common geoscience packages. See more information on how to run Jupyter notebooks at DKRZ [here](https://www.dkrz.de/up/systems/mistral/programming/jupyter-notebook). Find there how to run this Jupyter notebook in the DKRZ server out of the Jupyterhub, which will entail that you create the environment accounting for the required package dependencies. Running this Jupyter notebook in your premise, which is also known as [client-side](https://en.wikipedia.org/wiki/Client-side) computing, will also require that you install the necessary packages on you own but it will anyway fail because you will not have direct access to the data pool. Direct access to the data pool is one of the main benefits of the [server-side](https://en.wikipedia.org/wiki/Server-side) data-near computing we demonstrate in this use case. 

Thanks to the data and computer scientists Marco Kulüke, Fabian Wachsmann, Regina Kwee-Hinzmann, Caroline Arnold, Felix Stiehler, Maria Moreno, and Stephan Kindermann at DKRZ for their contribution to this notebook.

In this use case you will learn the following:
- How to access a dataset from the DKRZ CMIP6 model data archive
- How to count the annual number of summer days for a particular geolocation using this model dataset
- How to visualize the results


You will use:
- [Intake](https://github.com/intake/intake) for finding the data in the catalog of the DKRZ archive <img src="https://avatars.githubusercontent.com/u/1469464?s=400&v=4" style="width: 75px;"> 
- [Xarray](http://xarray.pydata.org/en/stable/) for loading and processing the data <img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" style="width: 200px;">
- [Pandas](https://pandas.pydata.org/) for structuring the data  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png" style="width: 200px;">
- [hvPlot](https://hvplot.holoviz.org/index.html) for visualizing the data in the Jupyter notebook and save the plots on your local computer <img src="https://hvplot.holoviz.org/assets/hvplot-wm.png" style="width: 75px;"> 

## 0. Load Packages

In [1]:
import numpy as np                    # fundamental package for scientific computing
import pandas as pd                   # data analysis and manipulation tool
import xarray as xr                   # handling labelled multi-dimensional arrays
import intake                         # to find data in a catalog, this notebook explains how it works
from ipywidgets import widgets        # to use widgets in the Jupyer Notebook
from geopy.geocoders import Nominatim # Python client for several popular geocoding web services
import folium                         # visualization tool for maps
import hvplot.pandas                  # visualization tool for interactive plots
import fsspec                         # unify various projects and classes to work with remote filesystems andfile-system-like abstractions using a standard pythonic interface
import zarr
import intake_esm

## 1. Which dataset do we need? -> Place, and Year

<a id='selection'></a>

In [2]:
# Produce the widget where we can select what geolocation and year are interested on 

print("Feel free to change the default values.")

place_box = widgets.Text(description="Enter place:", value="Hamburg")
display(place_box)

year_box = widgets.Dropdown(options=range(2015, 2101), description="Select year: ", disabled=False,)
display(year_box)

Feel free to change the default values.


Text(value='Hamburg', description='Enter place:')

Dropdown(description='Select year: ', options=(2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 202…

### 1.1 Find Coordinates of chosen Place
If ambiguous, the most likely coordinates will be chosen, e.g. "Hamburg" results in "Hamburg, 20095, Deutschland", (53.55 North, 10.00 East)

In [3]:
# We use the module Nominatim, which gives us the geographical coordinates of the place we selected above

geolocator = Nominatim(user_agent="any_agent")
location = geolocator.geocode(place_box.value)

print(location.address)
print((location.latitude, location.longitude))

Hamburg, Deutschland
(53.550341, 10.000654)


### 1.2 Show Place on a Map

In [4]:
# We use the folium package to plot our selected geolocation in a map

m = folium.Map(location=[location.latitude, location.longitude])
tooltip = location.latitude, location.longitude
folium.Marker([location.latitude, location.longitude], tooltip=tooltip).add_to(m)
display(m)

We have defined the place and time. Now, we can search for the climate model dataset.

## 2. Intake Catalog
Similar to the shopping catalog at your favorite online bookstore, the intake catalog contains information (e.g. model, variables, and time range) about each dataset (the title, author, and number of pages of the book, for instance) that you can access before loading the data. It means that thanks to the catalog, you can find where is the book just by using some keywords and you do not need to hold it in your hand to know the number of pages, for instance.

### 2.1 Load the Intake Catalog
We load the catalog descriptor with the intake package. The catalog is updated daily. The catalog descriptor is created by the DKRZ developers that manage the catalog, you do not need to care so much about it, knowing where it is and loading it is enough:

In [5]:
# Open the catalog with the intake package and name it "col" as short for "collection"
col=intake.open_esm_datastore("https://gitlab.dkrz.de/data-infrastructure-services/intake-esm/-/raw/master/esm-collections/cloud-access/dkrz_cmip6_disk.json")

/opt/conda/lib/python3.12/site-packages/intake_esm/cat.py:249: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [6]:
col.df.columns

Index(['activity_id', 'institution_id', 'source_id', 'experiment_id',
       'member_id', 'table_id', 'variable_id', 'grid_label', 'dcpp_init_year',
       'version', 'time_range', 'path', 'opendap_url', 'project',
       'simulation_id', 'grid_id', 'frequency', 'time_reduction', 'long_name',
       'units', 'realm', 'level_type', 'time_min', 'time_max', 'format',
       'uri'],
      dtype='object')

Let's see what is inside the intake catalog. The underlying data base is given as a pandas dataframe which we can access with "col.df". Then, "col.df.head()" shows us the first rows of the table of the catalog.

This catalog contains all datasets of the CMIP6 archive at DKRZ. In the next step we narrow the results down by chosing a model and variable.

### 2.2 Browse the Intake Catalog
In this example we chose the Max-Planck Earth System Model in High Resolution Mode ("MPI-ESM1-2-HR") and the maximum temperature near surface ("tasmax") as variable. We also choose an experiment. CMIP6 comprises several kind of experiments. Each experiment has various simulation members. you can find more information in the [CMIP6 Model and Experiment Documentation](https://pcmdi.llnl.gov/CMIP6/Guide/dataUsers.html#5-model-and-experiment-documentation).

In [7]:
# Store the name of the model we chose in a variable named "climate_model"

climate_model = "MPI-ESM1-2-HR" # here we choose Max-Plack Institute's Earth Sytem Model in high resolution

# This is how we tell intake what data we want

query = dict(
    source_id      = climate_model, # the model
#    experiment_id  = "ssp370",
    member_id      = "r1i1p1f1",
    variable_id    = "tasmax", # temperature at surface, maximum
    table_id       = "day", # daily maximum
)

# Intake looks for the query we just defined in the catalog of the CMIP6 data pool at DKRZ
cat = col.search(**query)

# Show query results
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,frequency,time_reduction,long_name,units,realm,level_type,time_min,time_max,format,uri
0,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18500101.0,18541231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
1,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18550101.0,18591231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
2,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18600101.0,18641231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
3,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18650101.0,18691231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
4,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18700101.0,18741231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp585,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,20800101.0,20841231,netcdf,/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ...
649,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp585,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,20850101.0,20891231,netcdf,/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ...
650,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp585,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,20900101.0,20941231,netcdf,/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ...
651,ScenarioMIP,DKRZ,MPI-ESM1-2-HR,ssp585,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,day,maximum,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,20950101.0,20991231,netcdf,/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ...


The result of the query are like the list of results you get when you search for articles in the internet by writing  keywords in your search engine (Duck duck go, Ecosia, Google,...). Thanks to the intake package, we did not need to know the path of each dataset, just selecting some keywords (the model name, the variable,...) was enough to obtain the results. If advance users are still interested in the location of the data inside the DKRZ archive, intake also provides the path and the OpenDAP URL (see the last columns above). 


Now we will find which file in the dataset contains our selected year so in the next section we can just load that specific file and not the whole dataset.

### 2.3 Find the Dataset That Contains the Year You Selected in Drop Down Menu Above

In [8]:
# Create a copy of cat.df, thus further modifications do not affect it

query_result_df = cat.df.copy() # new dataframe


# Each dataset contains many files, extract the initial and final year of each file

query_result_df["start_year"] = query_result_df["time_range"].str[0:4].astype(int) # add column with start year
query_result_df["end_year"] = query_result_df["time_range"].str[9:13].astype(int) # add column with end year


# Delete the time range column

query_result_df.drop(columns=["time_range"], inplace = True) # "inplace = False" will drop the column in the view but not in the actual dataframe
query_result_df.iloc[0:3]

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,long_name,units,realm,level_type,time_min,time_max,format,uri,start_year,end_year
0,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18500101.0,18541231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...,1850,1854
1,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18550101.0,18591231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...,1855,1859
2,CMIP,MPI-M,MPI-ESM1-2-HR,1pctCO2,r1i1p1f1,day,tasmax,gn,NaN,v20190710,...,Daily Maximum Near-Surface Air Temperature,K,atmos,NaN,18600101.0,18641231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/MPI-M/MPI-E...,1860,1864


In [9]:
# Select the file that contains the year we selected in the drop down menu above, e.g. 2015

selected_file = query_result_df[(year_box.value >= query_result_df["start_year"]) & (
                   year_box.value <= query_result_df["end_year"])]


# Path of the file that contains the selected year

selected_path = selected_file["opendap_url"].values[0] 


# Show the path of the file that contains the selected year

selected_path

'http://esgf3.dkrz.de/thredds/dodsC/cmip6/CMIP/MPI-M/MPI-ESM1-2-HR/piControl/r1i1p1f1/day/tasmax/gn/v20190710/tasmax_day_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn_20150101-20191231.nc'

In [10]:
# This script dynamically constructs fileServer URLs and fetches data for loading into the model.
import os
import requests
import xarray as xr

# Define the HTTP download function
def download_data(url, save_path):
    """Downloads data from an HTTP URL and saves it to a specified path."""
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {save_path}")
    else:
        raise Exception(f"Failed to download data. Status code: {response.status_code}")

# Function to fetch and load data based on the selected year
def fetch_and_load_data(selected_file):
    """Fetches the data file for the selected year and loads it."""
    # Extract the OPeNDAP URL
    opendap_url = selected_file["opendap_url"].values[0]

    # Construct the fileServer URL from the OPeNDAP URL
    file_server_url = opendap_url.replace("dodsC", "fileServer")

    # Extract the file name from the URL
    save_file = os.path.basename(file_server_url)

    # Download the file
    download_data(file_server_url, save_file)

    # Load the data using xarray
    try:
        ds = xr.open_dataset(save_file)
        print("Dataset loaded successfully.")
        return ds
    except Exception as e:
        raise Exception(f"Error loading dataset: {e}")

# Example selection logic for the year
selected_file = query_result_df[(year_box.value >= query_result_df["start_year"]) & (
                   year_box.value <= query_result_df["end_year"])]

if not selected_file.empty:
    # Fetch and load the dataset
    ds_tasmax = fetch_and_load_data(selected_file)

    # Open variable "tasmax" over the whole time range
    tasmax_xr = ds_tasmax["tasmax"]

    # Define start and end time string
    time_start = str(year_box.value) + "-01-01T12:00:00.000000000"
    time_end = str(year_box.value) + "-12-31T12:00:00.000000000"

    # Slice selected year
    tasmax_year_xr = tasmax_xr.loc[time_start:time_end, :, :]
    print("Data for the selected year sliced successfully.")
else:
    print("No data file available for the selected year.")


Downloaded: tasmax_day_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn_20150101-20191231.nc
Dataset loaded successfully.
Data for the selected year sliced successfully.


## 3. Load the model data

In [11]:
# Let's have a look at the xarray data array

tasmax_year_xr

<xarray.DataArray 'tasmax' (time: 365, lat: 192, lon: 384)> Size: 108MB
[26910720 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 3kB 2015-01-01T12:00:00 ... 2015-12-31T12:...
  * lat      (lat) float64 2kB -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon      (lon) float64 3kB 0.0 0.9375 1.875 2.812 ... 357.2 358.1 359.1
    height   float64 8B ...
Attributes:
    standard_name:  air_temperature
    long_name:      Daily Maximum Near-Surface Air Temperature
    comment:        maximum near-surface (usually, 2 meter) air temperature (...
    units:          K
    cell_methods:   area: mean time: maximum
    cell_measures:  area: areacella
    history:        2019-08-14T18:59:51Z altered by CMOR: Treated scalar dime...

We see not only the numbers, but also information about it, such as long name, units, and the data history. This information is called metadata.

## 4. Compare Model Grid Cell with chosen Location

In [12]:
# Find nearest model coordinate by finding the index of the nearest grid point

abslat = np.abs(tasmax_year_xr["lat"] - location.latitude)
abslon = np.abs(tasmax_year_xr["lon"] - location.longitude)
c = np.maximum(abslon, abslat)

([xloc], [yloc]) = np.where(c == np.min(c)) # xloc and yloc are the indices of the neares model grid point

In [13]:
# Draw map again

m = folium.Map(location=[location.latitude, location.longitude], zoom_start=8)


tooltip = location.latitude, location.longitude
folium.Marker(
    [location.latitude, location.longitude],
    tooltip=tooltip,
    popup="Location selected by You",
).add_to(m)


tooltip = float(tasmax_year_xr["lat"][yloc].values), float(tasmax_year_xr["lon"][xloc].values)
folium.Marker(
    [tasmax_year_xr["lat"][yloc], tasmax_year_xr["lon"][xloc]],
    tooltip=tooltip,
    popup="Model Grid Cell Center",
).add_to(m)


# Define coordinates of model grid cell (just for visualization)

rect_lat1_model = (tasmax_year_xr["lat"][yloc - 1] + tasmax_year_xr["lat"][yloc]) / 2
rect_lon1_model = (tasmax_year_xr["lon"][xloc - 1] + tasmax_year_xr["lon"][xloc]) / 2
rect_lat2_model = (tasmax_year_xr["lat"][yloc + 1] + tasmax_year_xr["lat"][yloc]) / 2
rect_lon2_model = (tasmax_year_xr["lon"][xloc + 1] + tasmax_year_xr["lon"][xloc]) / 2


# Draw model grid cell

folium.Rectangle(
    bounds=[[rect_lat1_model, rect_lon1_model], [rect_lat2_model, rect_lon2_model]],
    color="#ff7800",
    fill=True,
    fill_color="#ffff00",
    fill_opacity=0.2,
).add_to(m)

m

Climate models have a finite resolution. Hence, models do not provide the data of a particular point, but the mean over a model grid cell. Take this in mind when comparing model data with observed data (e.g. weather stations).


Now, we will visualize the daily maximum temperature time series of the model grid cell.

## 5. Draw Temperature Time Series and Count Summer days

The definition of a summer day varies from region to region. According to the [German Weather Service](https://www.dwd.de/EN/ourservices/germanclimateatlas/explanations/elements/_functions/faqkarussel/sommertage.html), "a summer day is a day on which the maximum air temperature is at least 25.0°C". Depending on the place you selected, you might want to apply a different threshold to calculate the summer days index. 

In [14]:
tasmax_year_place_xr = tasmax_year_xr[:, yloc, xloc] - 273.15 # Convert Kelvin to °C
tasmax_year_place_df = pd.DataFrame(index = tasmax_year_place_xr['time'].values, 
                                    columns = ['Model Temperature', 'Summer Day Threshold']) # create the dataframe

tasmax_year_place_df.loc[:, 'Model Temperature'] = tasmax_year_place_xr.values # insert model data into the dataframe
tasmax_year_place_df.loc[:, 'Summer Day Threshold'] = 25 # insert the threshold into the dataframe


# Plot data and define title and legend

tasmax_year_place_df.hvplot.line(y=['Model Temperature', 'Summer Day Threshold'], 
                                 value_label='Temperature in °C', legend='bottom', 
                                 title='Daily maximum Temperature near Surface for '+place_box.value, 
                                 height=500, width=620)

:NdOverlay   [Variable]
   :Curve   [index]   (Temperature in °C)

As we can see, the maximum daily temperature is highly variable over the year. As we are using the mean temperature in a model grid cell, the amount of summer days might we different that what you would expect at a single location.

In [15]:
# Summer days index calculation

no_summer_days_model = tasmax_year_place_df["Model Temperature"][tasmax_year_place_df["Model Temperature"] > 25].size # count the number of summer days


# Print results in a sentence

print("According to the German Weather Service definition, in the ssp370 scenario the " 
      +climate_model +" model shows " +str(no_summer_days_model) +" summer days for " +str(place_box.value) 
      + " in " + str(year_box.value) +".")

According to the German Weather Service definition, in the ssp370 scenario the MPI-ESM1-2-HR model shows 3 summer days for Hamburg in 2015.


[Try another location and year](#selection)